# 15차시: [실습] 네이버 금융에서 뉴스 타이틀과 시장 지표 크롤링

## 학습 목표
- 네이버 금융 페이지의 HTML 구조를 분석하는 방법 학습
- 주요 시장 지표(환율, 유가, 지수)를 크롤링하는 실습
- 뉴스 헤드라인과 링크를 수집하는 실습

## 학습 내용
1. 네이버 금융 페이지 구조 분석
2. 시장 지표 크롤링 (환율, 유가, 지수)
3. 뉴스 헤드라인 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
import time
from IPython.display import display

In [2]:
# 공통 설정
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

def get_soup(url):
    """URL에서 BeautifulSoup 객체 반환"""
    response = requests.get(url, headers=HEADERS, timeout=10)
    response.raise_for_status()
    return BeautifulSoup(response.text, 'html.parser')

---
## 1. 네이버 금융 페이지 구조 분석

네이버 금융 메인 페이지: https://finance.naver.com/

### 주요 섹션
- **시세 정보**: 코스피, 코스닥, 환율, 유가 등
- **뉴스**: 증권 뉴스 헤드라인
- **인기 종목**: 거래량/상승률 상위 종목

---
## 2. 시장 지표 크롤링

### 2.1 KOSPI/KOSDAQ 지수

In [8]:
url = "https://finance.naver.com/sise/sise_index.naver?code=KOSPI"

# url = "https://finance.naver.com/sise/sise_index.naver?code=KOSDAQ"

soup = get_soup(url)

# 현재가
"""
#now_value
"""
now_value = soup.select_one('#now_value')
print(now_value)

# 증감
"""
#change_value_and_rate > span
"""
fluc = soup.select_one('#change_value_and_rate > span')
print(fluc)

current = now_value.get_text(strip=True)
change = fluc.get_text(strip=True)

print("현재가:", current)
print("증감:", change)

<em id="now_value">4,581.46</em>
<span>29.09</span>
현재가: 4,581.46
증감: 29.09


### 2.2 환율 크롤링

In [12]:
# 환율 크롤링
print("[환율 크롤링]")
print("=" * 60)

# 네이버 금융 환율 페이지
url = "https://finance.naver.com/marketindex/"
soup = get_soup(url)

# 환율 정보 추출
"""
#exchangeList > li.on > a.head.usd > div > span.value
"""
exchange = soup.select('#exchangeList > li.on > a.head.usd > div > span.value')
print(exchange)

print(exchange[0].get_text(strip=True))

[환율 크롤링]
[<span class="value">1,455.90</span>]
1,455.90


---
## 3. 뉴스 헤드라인 크롤링

In [25]:
url = "https://finance.naver.com/news/mainnews.naver"
soup = get_soup(url)

news_data = []
crawl_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# 뉴스 리스트
# <ul class="newsList"> 안의 <li> 요소를 모두 출력
print(soup.select('ul.newsList li')[:2])

[<li class="block1">
<dl>
<dt class="thumb">
<a href="/news/news_read.naver?article_id=0001135200&amp;office_id=366&amp;mode=mainnews&amp;type=&amp;date=2026-01-09&amp;page=1"><img onerror="this.src='https://ssl.pstatic.net/static/nfinance/2017/02/27/thumb_72x54.gif'" src="https://imgnews.pstatic.net/image/thumb70/366/2026/01/09/1135200.jpg"/></a>
</dt>
<dd class="articleSubject">
<a href="/news/news_read.naver?article_id=0001135200&amp;office_id=366&amp;mode=mainnews&amp;type=&amp;date=2026-01-09&amp;page=1">“300달러까지 빠진다” 경고에도... 서학개미, 새해 테슬라 1조 베팅</a>
</dd>
<dd class="articleSummary">
										국내 투자자, 올해 테슬라 5400억원 순매수 관련 레버리지 ETF도 4000억원 넘게 사들여 테슬라가 실적 부진으로 약세를 면치 못하..
										<span class="press">조선비즈 </span>
<span class="bar">|</span>
<span class="wdate">2026-01-09 12:41:11</span>
</dd>
</dl>
</li>, <li class="block1">
<dl>
<dt class="thumb">
<a href="/news/news_read.naver?article_id=0000461022&amp;office_id=629&amp;mode=mainnews&amp;type=&amp;date=2026-01-09&amp;page=1"><img on

In [26]:
def crawl_financial_news(limit=10):
    """
    네이버 금융 주요 뉴스 헤드라인 크롤링
    """

    url = "https://finance.naver.com/news/mainnews.naver"
    soup = get_soup(url)

    news_data = []
    crawl_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # 뉴스 리스트
    news_items = soup.select("ul.newsList li")

    for item in news_items[:limit]:
        try:
            # (1) 뉴스 제목 및 링크 추출
            title_elem = item.select_one("dd.articleSubject a")   #  dd.articleSubject 안의 a 태그
            if not title_elem:
                continue

            # title 속성이 있으면 우선 사용, 없으면 a 태그의 텍스트를 사용
            title = title_elem.get("title", title_elem.get_text(strip=True))
            # 기사 링크 추출
            link = title_elem.get("href", "")
            # 상대경로일 경우 절대경로로 변환
            if link.startswith("/"):
                link = "https://finance.naver.com" + link

            # (2) 언론사 정보 추출. class="press" 요소
            press_elem = item.select_one(".press")
            press = press_elem.get_text(strip=True) if press_elem else "N/A"

            news_data.append({
                "제목": title,
                "출처": press,
                "링크": link,
                "수집시각": crawl_time
            })

        except Exception:
            continue

    return pd.DataFrame(news_data)

In [27]:
from IPython.display import HTML, display

# 함수 테스트
print("[뉴스 크롤링 함수 테스트]")
print("=" * 60)
df_news_test = crawl_financial_news(limit=5)
print(f"수집된 뉴스: {len(df_news_test)}건\n")

if not df_news_test.empty:
    df_show = df_news_test.copy()

    # 제목을 클릭 가능한 링크(HTML)로 변환
    df_show["제목"] = df_show.apply(
        lambda row: f'<a href="{row["링크"]}" target="_blank">{row["제목"]}</a>',
        axis=1
    )

    # 링크 컬럼은 숨기고(원하면 유지 가능)
    df_show = df_show[["제목", "출처", "수집시각"]]

    # HTML로 표시 (escape=False 중요!)
    display(HTML(df_show.to_html(escape=False, index=False)))
else:
    print("❌ 수집된 뉴스가 없습니다.")

[뉴스 크롤링 함수 테스트]
수집된 뉴스: 5건



제목,출처,수집시각
"“300달러까지 빠진다” 경고에도... 서학개미, 새해 테슬라 1조 베팅",조선비즈,2026-01-09 04:10:09
국장은 불장인데…얼어붙은 2차전지주에 개미들 '울상',더팩트,2026-01-09 04:10:09
아이온 이어 ‘나쁜게임’ 리니지도 돌아온다…NC의 부활? “영업익 1300억 돌파할 것” [종목Pick],헤럴드경제,2026-01-09 04:10:09
트럼프 '꿈의 군대' 기대감에 방산株 돌격…한화에어로 신고가,머니투데이,2026-01-09 04:10:09
"""그때 더 살걸""…주춤하던 비트코인, 다시 위로 고개",이코노미스트,2026-01-09 04:10:09


---
## 학습 정리

### 네이버 금융 크롤링 URL
| 페이지 | URL |
|--------|-----|
| 메인 | https://finance.naver.com/ |
| 시장지표 | https://finance.naver.com/marketindex/ |
| 주요뉴스 | https://finance.naver.com/news/mainnews.naver |

---

### 다음 차시 예고
- 16차시: 크롤링 데이터 정제 및 SQLite 저장
  - Pandas로 데이터 정제
  - SQLite 데이터베이스 기초
  - 테이블 생성 및 데이터 저장/조회